### A simple A <-> B reaction between 2 species,
with 1st-order kinetics in both directions, taken to equilibrium

Based on the experiment _"1D/reactions/reaction_1"_ ; this is simply the "single-compartment" version of it.

LAST REVISED: Dec. 4, 2022

In [1]:
# Extend the sys.path variable, to contain the project's root directory
import set_path
set_path.add_ancestor_dir_to_syspath(2)  # The number of levels to go up 
                                         # to reach the project's home, from the folder containing this notebook

Added 'D:\Docs\- MY CODE\BioSimulations\life123-Win7' to sys.path


In [2]:
from experiments.get_notebook_info import get_notebook_basename

from modules.reactions.reaction_data import ReactionData as chem
from modules.reactions.reaction_dynamics import ReactionDynamics

import numpy as np
import plotly.express as px

In [3]:
# Initialize the reaction
chem_data = chem(names=["A", "B"])

# Reaction A <-> B , with 1st-order kinetics in both directions
chem_data.add_reaction(reactants=["A"], products=["B"], forward_rate=3., reverse_rate=2.)

print("Number of reactions: ", chem_data.number_of_reactions())

Number of reactions:  1


In [4]:
chem_data.describe_reactions()

Number of reactions:  1
0: A <-> B  (kF = 3.0 / kR = 2.0)


In [5]:
dynamics = ReactionDynamics(reaction_data=chem_data)

In [6]:
# Initial concentrations of all the chemicals, in index order
dynamics.set_conc([10., 50.])

In [7]:
dynamics.describe_state()

SYSTEM STATE at Time t = 0.0:
2 species:
  Species 0 (A).Conc: 10.0
  Species 1 (B).Conc: 50.0


In [9]:
dynamics.add_snapshot(caption="Initial state")

In [10]:
dynamics.history.get()

,SYSTEM TIME,A,B,caption
0,0.0,10.0,50.0,Initial state


In [6]:
# First step of reaction
dynamics.single_compartment_react(time_step=0.1, n_steps=1,
                                  snapshots=None)

In [7]:
# Numerous more steps
dynamics.single_compartment_react(time_step=0.1, n_steps=10,
                                  snapshots=None)

In [8]:
dynamics.get_conc(form="DICT")

{'A': 23.9931640625, 'B': 36.0068359375}

NOTE: Consistent with the 3/2 ratio of forward/reverse rates (and the 1st order reactions),
 the systems settles in the following equilibrium:

[A] = 23.99316406
 
[B] = 36.00683594


In [9]:
dynamics.is_in_equilibrium(rxn_index=0, conc=dynamics.get_conc(form="DICT"))

Ratio of forward/reverse reaction rates: 1.5
Ratio of reactant/product concentrations, adjusted for reaction orders: 1.5007122797020636
    [B] / [A]


True